In [49]:
import cv2
from ultralytics import YOLO
from ultralytics.engine.results import Results

# Load a pretrained YOLOv8n model
model = YOLO('yolov8x-seg.pt')

# Run inference on an image
results: Results = model('예제선반1.jpg')  # results list


image 1/1 c:\Jupyter Notebook\Project_NewBie\1.jpg: 480x640 4 bottles, 1 cup, 1144.5ms
Speed: 3.5ms preprocess, 1144.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)


모델을 불러오는 과정

In [51]:
# Global variables
drawing = False # true if mouse is pressed
ix, iy = -1, -1
img = None
temp_img = None

# Mouse callback function
def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, img, temp_img

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
        temp_img = img.copy()

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing is True:
            img = temp_img.copy()
            cv2.rectangle(img, (ix, iy), (x, y), (0, 255, 0), 1)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.rectangle(img, (ix, iy), (x, y), (0, 255, 0), -1)
        width = abs(ix - x)
        height = abs(iy - y)
        print(f"Width: {width}, Height: {height}")

# Load an image
img = results[0].orig_img.copy()
cv2.namedWindow('image')
cv2.setMouseCallback('image', draw_rectangle)

while(1):
    cv2.imshow('image', img)
    if cv2.waitKey(1) & 0xFF == 27:  # exit if ESC is pressed
        break

cv2.destroyAllWindows()

Width: 222, Height: 153
Width: 242, Height: 159


마우스 드래그로 사각형을 그리는 기능

In [52]:
image_path = '예제선반1.jpg'
image = cv2.imread(image_path)

for r in results:
    for idx, box in enumerate(r.boxes.xyxy):
        x1, y1, x2, y2 = map(int, box[:4])  # 경계 상자 좌표 (정수로 변환)
        cropped_img = image[y1:y2, x1:x2]  # 경계 상자 좌표를 사용하여 이미지 자르기

        height, width = cropped_img.shape[:2]
        new_width = int(width)
        new_height = int(height)
        cropped_img = cv2.resize(cropped_img, (new_width, new_height))
            
        print(f'Detected Object {idx+1} width: {new_width}, height: {new_height}')  # 객체의 가로와 세로 길이 출력
        cv2.imshow(f'Detected Object {idx+1}', cropped_img)
        cv2.waitKey(0)

cv2.destroyAllWindows() # 이미지창 닫기

Detected Object 1 width: 112, height: 322
Detected Object 2 width: 125, height: 431
Detected Object 3 width: 110, height: 173
Detected Object 4 width: 138, height: 460
Detected Object 5 width: 54, height: 199


객체를 하나씩 보여주고 가로 세로 길이를 출력하는 기능